In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
import torchvision
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
import visdom

vis = visdom.Visdom()
vis.close(env="main")

Setting up a new session...


''

In [2]:
## hyperparameter

device = 'cuda' if torch.cuda.is_available() else 'cpu'


batch_size =10
learning_rate = 1e-3
num_epochs =25


In [3]:
## Transform

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224,224)),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])
train_data = datasets.ImageFolder('.\\archive\\Training_Crop_Aug\\',  transform = transform)
train_dataloader = torch.utils.data.DataLoader(train_data,batch_size = batch_size,shuffle = True)

test_data = datasets.ImageFolder('.\\archive\\Testing_Crop\\',  transform = transform)
test_dataloader = torch.utils.data.DataLoader(test_data,batch_size = batch_size,shuffle = True)

for num, value in enumerate(test_data):
    data, label =value
    #print(num,label)
    #print(data.shape)


In [4]:
## CNN

class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.layer1 = nn.Sequential(
            #224-5 = 219/1+1 =220
            nn.Conv2d(3,8,5),
            nn.SELU(),
            nn.MaxPool2d(2)
        )
        self.layer2 = nn.Sequential(
            # 110-5 = 105/1 +1 = 106
            nn.Conv2d(8,32,5),
            nn.SELU(),
            nn.MaxPool2d(2)
        )
        self.layer3 = nn.Sequential(
            #53-4 = 49/1+1 =50
            nn.Conv2d(32,64,4),
            nn.SELU(),
        )
        self.fc = nn.Linear(64*50*50,4, bias = True)
        torch.nn.init.xavier_uniform_(self.fc.weight)
        '''
        self.dropout = nn.Dropout(0.3)
        self.conv1 = nn.Conv2d(3,8,5,stride=1)
        self.conv2 = nn.Conv2d(8,32,5,stride = 1)
        self.conv3 = nn.Conv2d(32,128,3,stride = 1,padding = 1)
        self.pool = nn.MaxPool2d(2)
        self.flat = nn.Flatten()
        self.fc1 = nn.Linear(128*15*15,1024)
        self.fc2 = nn.Linear(1024,128)
        self.fc3 = nn.Linear(128,4)
        '''
    def forward(self,x):
        out = self.layer1(x)
        #print(out.shape)
        out = self.layer2(out)
        #print(out.shape)
        out = self.layer3(out)
        out = out.view(out.size(0),-1)
        #print(out.shape)
        out = self.fc(out)
        return out

In [5]:
model = CNN().to(device)
A = (torch.Tensor(10,3,224,224)).to(device)
test_out = model(A)
model

C:\Users\jk121\Anaconda3\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


CNN(
  (layer1): Sequential(
    (0): Conv2d(3, 8, kernel_size=(5, 5), stride=(1, 1))
    (1): SELU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(8, 32, kernel_size=(5, 5), stride=(1, 1))
    (1): SELU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(4, 4), stride=(1, 1))
    (1): SELU()
  )
  (fc): Linear(in_features=160000, out_features=4, bias=True)
)

In [11]:
class resNet(nn.Module):
    

SyntaxError: unexpected EOF while parsing (<ipython-input-11-78367677fe9e>, line 2)

In [6]:
criterion = nn.CrossEntropyLoss().to(device)

#Use Adam optimizer(We tried a lot of optimizers but results are similar)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [7]:


def loss_tracker(loss_plot, loss_value, num):
    '''num, loss_value, are Tensor'''
    vis.line(X=num,
             Y=loss_value,
             win = loss_plot,
             update='append'
             )

In [8]:

total_batch = len(train_dataloader)

for epoch in range(num_epochs):
    avg_cost = 0
    for images, labels in train_dataloader:
        #Push tensors to GPU
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(images)

        cost = criterion(hypothesis, labels)
        cost.backward()      
        optimizer.step()
        
        avg_cost += cost / total_batch
    
    print('[Epoch:{}] cost = {}'.format(epoch+1, avg_cost))
    #loss_tracker(loss_plt, torch.Tensor([avg_cost]), torch.Tensor([epoch]))

[Epoch:1] cost = 1.347277283668518
[Epoch:2] cost = 0.6886361241340637
[Epoch:3] cost = 0.5561000108718872
[Epoch:4] cost = 0.390829861164093
[Epoch:5] cost = 16.00760269165039
[Epoch:6] cost = 0.2159266322851181
[Epoch:7] cost = 0.049353860318660736
[Epoch:8] cost = 0.03183635324239731
[Epoch:9] cost = 0.012724313884973526
[Epoch:10] cost = 0.016719359904527664
[Epoch:11] cost = 0.20160438120365143
[Epoch:12] cost = 0.4869080185890198
[Epoch:13] cost = 0.19351257383823395
[Epoch:14] cost = 0.21554459631443024
[Epoch:15] cost = 0.8077020645141602
[Epoch:16] cost = 1.1340596675872803
[Epoch:17] cost = 0.41412001848220825
[Epoch:18] cost = 0.3177545964717865
[Epoch:19] cost = 1.723042368888855
[Epoch:20] cost = 0.4813148081302643
[Epoch:21] cost = 0.37729310989379883
[Epoch:22] cost = 3.3122379779815674
[Epoch:23] cost = 0.6940370798110962
[Epoch:24] cost = 0.23049010336399078
[Epoch:25] cost = 0.25781679153442383


In [10]:
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_dataloader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))

Accuracy of the model on the test images: 76.90355329949239 %


In [ ]:
start = time.time()           #시작 time 재기!
history = []
best_acc = 0.0                #Best Accuracy Value

  for epoch in range(epochs):
    epoch_start = time.time()
    print("Epoch: {}/{}".format(epoch+1, epochs))

    #Model to Training Mode!
    model.train()

    #Initialize Loss and Accuracy
    train_loss = 0.0
    train_acc = 0.0 

    for i, (inputs, labels) in enumerate(train_data_loader):
          inputs = inputs.to(device)
          labels = labels.to(device)

          optimizer.zero_grad()
          #Forward Pass
          outputs = model(inputs)

          #Loss Computation
          loss = loss_criterion(outputs, labels)

          #Back Propagation
          loss.backward()

          #Update the parameters
          optimizer.step()

        train_loss += loss.item() * inputs.size(0)

      #Computing the accuracy(다소 헷갈리는 파트, 다시 확인!!!)
      ret, predictions = torch.max(outputs.data, 1)
      correct_counts = predictions.eq(labels.data.view_as(predictions))

      acc = torch.mean(correct_counts.type(torch.FloatTensor))

          train_acc += acc.item() * inputs.size(0)

            if i % 50 == 0 and i != 0:
            print("Batch Number: {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}".format(i, loss.item(), acc.item()))

    avg_train_loss = train_loss / train_data_size
    avg_train_acc = train_acc / train_data_size
    history.append([avg_train_loss, avg_train_acc])

    epoch_end = time.time()

    print("Epoch : {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}%, Time: {:.4f}s".format(epoch, avg_train_loss, avg_train_acc*100, epoch_end-epoch_start))


In [209]:
###### need to check single img ######
"""
import cv2
import matplotlib.image as mpimg
im = cv2.imread('C:\\Users\\jk121\\Desktop\\temp\\archive\\Training\\p(1).jpg')



def test_one_image(I, model):
    '''
    I - 28x28 uint8 numpy array
    '''

    # test phase
    model.eval()

    # convert image to torch tensor and add batch dim
    batch = torch.tensor(I / 255).unsqueeze(0)

    # We don't need gradients for test, so wrap in 
    # no_grad to save memory
    with torch.no_grad():
        batch = batch.to(device)

        # forward propagation
        output = model( batch )

        # get prediction
        output = torch.argmax(output, 1)

    return output
"""